# Distributed Representations of Words and Phrases and their Compositionality

    Tomas Mikolov
    Ilya Sutskever
    Kai Chen
    Greg Corrado
    Jeffrey Dean

    2013 OCT 16

https://arxiv.org/pdf/1310.4546.pdf

## 总结
- The Skip-gram Model
    - 预测当前word（word representations）周围的word
    - 句子范围或段落范围
    - objective
        - 给定sequence $w_1,w_2,w_3,\cdots,w_T$
        - 最大化：$\frac{1}{T}\sum_{t=1}^T\sum_{-c \le j \le c,j \ne 0} log p(w_{t+j}|w_t)\ (1)$
        - c是上下文大小
    - 使用softmax定义$p(w_{t+j}|w_t)$
        - $p(w_O|w_I)=\frac{exp(v_{w_O}^{'T} v_{w_I})}{\sum_{w=1}^W exp(v_w^{'T} v_{w_I})}\ (2)$
        - W是vocabulary大小
- Hierarchical Softmax
    - 比一般的Softmax更高效，因为它只需要评估$log_2(W)$个节点
- Negative Sampling
    - Noise Contrastive Estimation (NCE)：最大化softmax的log probability
    - Negative sampling (NEG)与NCE区别是NCE同时需要抽样和 noise distribution的分布，Negative sampling只需要抽样
- Subsampling of Frequent Words
    - 平衡常见词和稀缺词的抽样方法

## Introduction
![1](http://ou8qjsj0m.bkt.clouddn.com//17-8-6/7181786.jpg)

## The Skip-gram Model
The training objective of the Skip-gram model is to find word representations that are useful for predicting the surrounding words in a sentence or a document. More formally, given a sequence of training words $w_1,w_2,w_3,\cdots,w_T$, the objective of the Skip-gram model is to maximize the average log probability

$\frac{1}{T}\sum_{t=1}^T\sum_{-c \le j \le c,j \ne 0} log p(w_{t+j}|w_t)\ (1)$

- c is the size of the training context (which can be a function of the center word $w_t$)

The basic Skip-gram formulation defines $p(w_{t+j}|w_t)$ using the softmax function:

$p(w_O|w_I)=\frac{exp(v_{w_O}^{'T} v_{w_I})}{\sum_{w=1}^W exp(v_w^{'T} v_{w_I})}\ (2)$

- $v_w$ and $v_w^{'}$ are the “input” and “output” vector representations of w
- W is the number of words in the vocabulary. 

### Hierarchical Softmax
A computationally efficient approximation of the full softmax is the `hierarchical softmax`.

The main advantage is that instead of evaluating W output nodes in the neural network to obtain the probability distribution, it is needed to evaluate only about $log_2(W)$ nodes.

Each word w can be reached by an appropriate path from the root of the tree:

- n(w,j) be the j-th node on the path from the root to w
- L(w) be the length of this path, so n(w, 1) = root and n(w, L(w)) = w
- for any inner node n, let ch(n) be an arbitrary fixed child of n and let [x] be 1 if x is true and -1 otherwise

$p(w|w_I)=\prod_{j=1}^{L(w)-1}\sigma([n(w,j+1)=ch(n(w,j))] \cdot {v'}_{n(w,j)}^T v_{w_I})\ (3)$

- $\sigma(x)=\frac{1}{1+exp(-x)}$
- $\sum_{w=1}^W p(w|w_I)=1$

Also, unlike the standard softmax formulation of the Skip-gram which assigns two representations $v_w$ and ${v'}_{w}$ to each word w, the hierarchical softmax formulation has one representation $v_w$ for each word w and one representation ${v'}_n$ for every inner node n of the binary tree.

### Negative Sampling
An alternative to the hierarchical softmax is Noise Contrastive Estimation (NCE). `NCE` posits that a good model should be able to differentiate data from noise by means of logistic regression.

While NCE can be shown to approximately maximize the log probability of the softmax, the Skip-gram model is only concerned with learning high-quality vector representations, so we are free to simplify NCE as long as the vector representations retain their quality. We define Negative sampling (NEG) by the objective:

$log\sigma({v'}_{w_O}^T v_{w_I})+\sum_{i=1}^k \mathbb{E}_{w_i \sim P_n(w)}[log \sigma(-{v'}_{w_i}^T v_{w_I})]\ (4)$

The main difference between the Negative sampling and NCE is that NCE needs both samples and the numerical probabilities of the noise distribution, while Negative sampling uses only samples. 

### Subsampling of Frequent Words
To counter the imbalance between the rare and frequent words, we used a simple subsampling approach: each word $w_i$ in the training set is discarded with probability computed by the formula

$P(w_i)=1-\sqrt{\frac{t}{f(w_i)}}\ (5)$

where $f(w_i)$ is the frequency of word wi and t is a chosen threshold, typically around $10^{-5}$.